In [1]:
import numpy as np
import shorttext

/Users/stephenhky/anaconda/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
# loading the whole ESV Bible
import biblebooks.bibledocs_iterator as bit
dbconn = bit.get_sqlite3_dbconn("ESVCrossway.sqlite")
biblecorpus = bit.generate_classdict_chapters(dbconn)

In [5]:
from biblebooks import BibleAbbrDict

In [7]:
# loading FastText model
wvmodel = shorttext.utils.load_fasttext_model('/Users/stephenhky/data/FastText/wiki.simple/wiki.simple.bin')

In [8]:
# define neural network

In [9]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Activation, Input
from keras.models import Sequential, Model
from keras.regularizers import l2

In [19]:
input_x = Input(shape=(None, wvmodel.vector_size))
output_y = LSTM(1024, return_sequences=True)(input_x)
print(output_y.shape)
output_y = Conv1D(1200, 120)(output_y)
print(output_y.shape)
output_y = MaxPooling1D(pool_size=100)(output_y)
print(output_y.shape)
output_y = Flatten()(output_y)
print(output_y.shape)
output_y = Dense(2, activation='sigmoid')(output_y)
print(output_y.shape)

kmodel = Model(inputs=input_x, outputs=output_y)

(?, ?, 1024)
(?, ?, 1200)
(?, ?, 1200)


ValueError: The shape of the input to "Flatten" is not fully defined (got (None, 1200). Make sure to pass a complete "input_shape" or "batch_input_shape" argument to the first layer in your model.

In [9]:
kmodel.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
wvmodel.vector_size

300

In [18]:
newdict = {}
newdict['ot'] = [sent for otbookabbr in BibleAbbrDict.otbookdict 
                 for chap in range(1, BibleAbbrDict.numchaps[otbookabbr]+1) 
                 for sent in biblecorpus[otbookabbr+'_'+str(chap)]]
newdict['nt'] = [sent for otbookabbr in BibleAbbrDict.ntbookdict 
                 for chap in range(1, BibleAbbrDict.numchaps[otbookabbr]+1) 
                 for sent in biblecorpus[otbookabbr+'_'+str(chap)]]

In [22]:
classifier = shorttext.classifiers.VarNNEmbeddedVecClassifier(wvmodel)

In [23]:
classifier.train(newdict, kmodel)

ValueError: Error when checking target: expected dense_1 to have 3 dimensions, but got array with shape (35885, 2)